In [1]:
from pathlib import Path
from typing import Literal

import pandas as pd
import geopandas as gpd
import numpy as np
import numpy.typing as npt
import rasterra as rt
import shapely
from shapely import wkt
import rra_tools
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from affine import Affine
from rasterio.fill import fillnodata
from scipy.signal import oaconvolve
import tqdm
import time
import os 
import PyPDF2
from PyPDF2 import PdfReader , PdfWriter, PdfMerger
from rra_tools import parallel

# Documentation
# testing change

## Inputs: 
- Fire perimeters for (1) continental US, (2) Hawaii, (3) Alaska
- Global Human Settlement Layer population estimates for 2000, 2005, 2010, 2015, 2020 (100m resolution, Molweide projection). Links in get_data.sh

## Analytic overview:

This script takes in a dataset of fire perimeters and a gridded population dataset. 

The parameters are: 
- Area threshold (area_thresh): this is the threshold by which we determine if a fire is large or small. Fires greater than or equal to 1000 acres are considered large. Based on size, fires get different buffers (one buffer for small fires and one for large). This is parameterized in square-meters.
- Large fire buffer (large_fire_buffer) = This is the buffer around the fire perimeter for large (>= 1000 acres) fires, in meters.
- Small fire buffer (small_fire_buffer) = This is the buffer around the fire perimeter for small (< 1000 acres) fires, in meters.
- Population averaging radius (pop_average_radius): This is the radius for a circle with the area that we are using in our denominator of population density. In other words, if our criteria is per 1 square-kilometer, this is the radius of a circle that has an area of 1 square-kilometer (or 1000 square-meters). This is parameterized in meters.
- Population density criteria (pop_density_criteria): This is the number of people per square-meter that are required to make an area a community.

The process is: 
1. It first buffers each fire, with a different buffer size based on whether the fire is big or small. A big fire is defined as a fire that is bigger than 1000 acres (converted to 4046856 meters). Big fires get buffered by 20km and small fires get buffered by 10km. 

2. Following the buffering of the fire, we create a bounding box around the fire that adds the radius of a `pop_average_radius` circle to each side of the fire. This is to ensure we pull enough of the population raster to do our density calculation correctly.

3. Once we have all of this information, we load a subset of our population raster using the bounding box that we created in step 2 to determine the subset loaded.

4. Using the population data, we build a kernel for the convolution - this is an average kernel.

5. We use our kernel to do a convolution -- for every pixel, we sum all the people within our radius of that pixel (currently 300m). The result of this is a raster of population density averaged to people per square kilometer.

6. Now, we can use our buffered fire perimeter to find the maximum value of population density for any individual pixel within the buffered perimeter. If any of them exceed our population density criteria (`pop_density_criteria`), then we determine that the fire overlaps with a community that exceeds our population density threshold and thus meets our population density criteria for the fire overlapping with a community. 


## Outputs: 
**Primary output**
- CSV with disaster_id and density_criteria_met variable
  
**Secondary outputs**
- Parquet file with metadata (disaster_id, density_criteria_met, state, year, buffer_distance, geometry, crs for that fire's utm)
- Diagnostic plots

## Next steps:
**To discuss with Milo**
- File of fires with no acreage (all_disaster_with_ics_poo_no_acreage_cont_us_select_vars) - have point location but not fire size.
- There are fires from puerto rico
  
**Coding next steps**
- put into executable python script and make error messages


In [2]:
data_dir = Path("~/Desktop/Desktop/epidemiology_PhD/00_repos/wildfire-disaster/data/").expanduser()
plot_dir = Path("~/Desktop/Desktop/epidemiology_PhD/02_projects/wildfire-disaster/plots/").expanduser()
# list(data_dir.glob("*.parquet"))
# list(data_dir.iterdir())
mol_crs = "ESRI: 54009"

In [5]:
# helper functions

def make_convolution_kernel(
    pixel_resolution_m: int | float,
    radius_m: int | float,
) :
    radius = int(radius_m // pixel_resolution_m)
    y, x = np.ogrid[-radius : radius + 1, -radius : radius + 1]

    kernel = (x**2 + y**2 < radius**2).astype(float)
    kernel = kernel / kernel.sum()
    return kernel


def make_spatial_average(
    tile: rt.RasterArray,
    radius: int | float,
) :
    arr = np.nan_to_num(tile.to_numpy())

    kernel = make_convolution_kernel(tile.x_resolution, radius) # tile.x_resolution is pulling the pop raster res, which is 100m in the ghsl data

    out_image = oaconvolve(arr, kernel, mode="same")

    out_raster = rt.RasterArray(
        out_image,
        transform=tile.transform,
        crs=tile.crs,
        no_data_value=tile.no_data_value,
    )
    return out_raster

1+1

2

In [4]:
# read in fires
fires_hi = gpd.read_parquet(data_dir / "raw/all_disaster_perimeters_buffers_hawaii_dist_select_vars.parquet")
fires_ak = gpd.read_parquet(data_dir / "raw/all_disaster_perimeters_buffers_alaska_dist_select_vars.parquet")
fires_conus = gpd.read_parquet(data_dir / "raw/all_disaster_perimeters_buffers_conus_dist_select_vars.parquet")
fires_misc = gpd.read_parquet(data_dir / "raw/all_disaster_with_ics_poo_no_acreage_cont_us_select_vars.parquet")

# read in utm map
utm_map = pd.read_csv(data_dir / "utm_popden.csv")
utm_map['state_list'] = utm_map['states'].apply(lambda s: s.split(','))
utm_map = utm_map.explode('state_list').set_index('state_list')['crs'].to_dict()
utm_map = {s.strip(): f"EPSG:{crs}" for s, crs in utm_map.items()}
utm_map["PR"] = "EPSG:3920"

FileNotFoundError: [Errno 2] Failed to open local file '/Users/laurenwilner/Desktop/Desktop/epidemiology_PhD/00_repos/wildfire-disaster/data/raw/all_disaster_perimeters_buffers_hawaii_dist_select_vars.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
# params
area_thresh = 4046856 # 1000 acres converted to sq meters
large_fire_buffer = 20000 # meters
small_fire_buffer = 10000 # meters
pop_average_radius = 300/np.sqrt(np.pi) # radius for a circle with area 300 sq m 
pop_density_criteria = 96 # people per sq km, which is 250 people per sq mile

fire_dfs = []
for df in [fires_conus, fires_ak, fires_hi]: 
    df['state'] = df['aggregate_states'].apply(lambda s: s[:2])
    df['year'] = df['aggregate_year'].apply(lambda y: round(y / 5)*5)
    keep_cols = ['disaster_id_clean', 'year', 'state', 'shape']
    row_tuples = list(df[keep_cols].itertuples(index=False, name=None))# [:5]
    for disaster_id, year, state, fire_poly in tqdm.tqdm(row_tuples):
        fire_crs = utm_map[state]
        fire_series = gpd.GeoSeries([fire_poly], crs=df.crs).to_crs(fire_crs)
        
        # I. buffer fire
         # i. convert fire to appropriate UTM
         # ii. calc area of fire poly
         # iii. create a perimeter around the fire 
           # large fire is if area > 20000
           # buffer_dist = ifelse(size == "large_fire", 20000, 10000))
         # iv. create a bounding box around that perimeter
           # buffer bounding box by half the radius of the kernel over which we are estimating density (so plus 1000/sqrt(pi))
         # v. convert the buffered bounding box to molweide projection
        buffer_dist = large_fire_buffer if fire_series.area.iloc[0] > area_thresh else small_fire_buffer 
        buffered_fire_series = fire_series.buffer(buffer_dist) # buffer dist in meters
        
        bounding_box = buffered_fire_series.envelope.buffer(pop_average_radius*1.1).to_crs(mol_crs).iloc[0]

        # II. load pop data 
        pop = rt.load_raster(data_dir / f"raw/pop_data/GHS_POP_E{year}_GLOBE_R2023A_54009_100_V1_0.tif", bounding_box).to_crs(fire_crs)
        pop_density_per_sq_km = pop * (1000**2 / pop.x_resolution**2)

        # III. build kernel and do convolution with kernel - for every pixel, sum all the people within a kilometer of that pixel,
          # so kernel should be all 1's and should be 10 pixels wide and 10 pixels tall
          # that convolution gives back a raster of pop density averaged to people per sq km
        mean_pop_density = make_spatial_average(pop_density_per_sq_km, pop_average_radius)

        # IV. determine if this fire meets density criteria by: 
          # i. take buffered fire poly and mask everything outside of that (set everything outside buffered poly to 0 which you can do w/ raster.mask)
          # ii. find max pixel val and if it exceeds your threshold then it overlaps with a communtiy that exceeds the threshold and is marked as TRUE in final csv. 
        max_pop_density = np.max(mean_pop_density.mask(buffered_fire_series).to_numpy())
        density_criteria_met = max_pop_density > pop_density_criteria

        # V. add to results df 
        df_fire = pd.DataFrame({
            'disaster_id': [disaster_id],
            'density_criteria_met': [density_criteria_met],
            'max_pop_density': [max_pop_density],
            'state': [state],
            'year': [year],
            'buffer_distance': [buffer_dist],
            'geometry': [fire_series.iloc[0]],
            'crs': [fire_crs],
            
        })
        fire_dfs.append(df_fire)

df = pd.concat(fire_dfs, ignore_index = True)
fires_included_prop = round(len(df[df["density_criteria_met"] == True])/len(df)*100, 2)
df['geometry'] = df['geometry'].apply(lambda geom: geom.wkt)
df.to_parquet(data_dir / "processed/fire_pop_density_criteria.parquet")
df[["disaster_id", "density_criteria_met"]].to_csv(data_dir / "processed/fire_pop_density_criteria.csv")

In [ ]:
df = pd.read_parquet(data_dir / "processed/fire_pop_density_criteria.parquet")
disaster_ids = df["disaster_id"]

In [ ]:
# plots 
def plot_fires(disaster_id): 
    area_thresh = 4046856 # 1000 acres converted to sq meters
    large_fire_buffer = 20000 # meters
    small_fire_buffer = 10000 # meters
    pop_average_radius = 300/np.sqrt(np.pi) # radius for a circle with area 1 sq km 
    pop_density_criteria = 96 # people per sq km, which is 250 people per sq mile
    
    cmap = "seismic"
    df_plot = pd.read_parquet(data_dir / "processed/fire_pop_density_criteria.parquet").iloc[0]
    df_plot = df[df["disaster_id"] == disaster_id].iloc[0]
    fire_poly = wkt.loads(df_plot["geometry"])
    crs = df_plot['crs']
    year = df_plot['year']
    
    fire_series = gpd.GeoSeries([fire_poly], crs=crs)
    buffer_dist = large_fire_buffer if fire_series.area.iloc[0] > area_thresh else small_fire_buffer 
    buffered_fire_series = fire_series.buffer(buffer_dist)
    fire_poly_plus_buffer = gpd.GeoSeries([fire_poly, buffered_fire_series.iloc[0]], crs=crs)
    bounding_box = buffered_fire_series.envelope.buffer(pop_density_radius * 1.1).to_crs(mol_crs).iloc[0]
    pop = rt.load_raster(data_dir / f"raw/pop_data/GHS_POP_E{year}_GLOBE_R2023A_54009_100_V1_0.tif", bounding_box).to_crs(crs)
    pop_density_per_sq_km = pop * (1000**2 / pop.x_resolution**2)
    mean_pop_density = make_spatial_average(pop_density_per_sq_km, pop_average_radius)
    
    # gridspec layout 
    fig = plt.figure(figsize=(15, 7))
    gs = gridspec.GridSpec(1, 2, figure=fig)
    
    # fig 1: fire poly, buffered fire poly, pop
    pop_transformed = pop*100/pop_density_criteria
    ax1 = fig.add_subplot(gs[0])
    pop_transformed.plot(ax=ax1, cmap = cmap, vmin = 1e-4, vmax = 2, under_color = "lightgrey")
    fire_poly_plus_buffer.boundary.plot(ax=ax1, linewidth=1.5, edgecolor = 'limegreen')
    ax1.set_title(f"Population counts", fontsize=16)
    ax1.set_axis_off()
    
    # fig 2: fire poly, buffered fire poly, pop density
    ax2 = fig.add_subplot(gs[1])
    pop_density_transformed = pop_density/pop_density_criteria
    pop_density_transformed.plot(ax=ax2, cmap = cmap, vmin = 1e-4, vmax = 2, under_color = "lightgrey")
    fire_poly_plus_buffer.boundary.plot(ax=ax2, linewidth=1.5, edgecolor = 'limegreen')
    ax2.set_title(f"Population density", fontsize=16)
    ax2.set_axis_off()
    
    fig.suptitle(f"Fire polygon with buffer (buffer distance: {buffer_dist} meters) \nDisaster ID: {df_plot['disaster_id']}, State: {df_plot['state']}, Year: {df_plot['year']} \nCriteria met: {df_plot["density_criteria_met"]}", fontsize=20)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    pdf_filename = plot_dir / f"fire_pop_dens_{df_plot['disaster_id']}.pdf"
    plt.savefig(pdf_filename, format='pdf')
    plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(15, 7))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], height_ratios=[2, 1])

df_fig = df[df["max_pop_density"]<100000]
# fig 1: histogram of max_pop_density for all fires
ax1 = fig.add_subplot(gs[:, 0])
ax1.hist(df_fig['max_pop_density'], bins=150, color='firebrick', edgecolor='black')
ax1.set_title('Histogram of maximum population density for all fires, subset to density < 100000')
ax1.set_xlabel('Max population density')
ax1.set_ylabel('Frequency')

# fig 2: histogram of max_pop_density where density_criteria_met = True
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(df_fig[df_fig['density_criteria_met'] == True]['max_pop_density'], bins=150, color='salmon', edgecolor='black')
ax2.set_title('Max population density, subset to density < 100000 (criteria met)')
ax2.set_xlabel('Max population density')
ax2.set_ylabel('Frequency')

# fig 3: histogram of max_pop_density where density_criteria_met = False
ax3 = fig.add_subplot(gs[1, 1])
ax3.hist(df_fig[df_fig['density_criteria_met'] == False]['max_pop_density'], bins=150, color='peachpuff', edgecolor='black')
ax3.set_title('Max population density (criteria not met)')
ax3.set_xlabel('Max population density')
ax3.set_ylabel('Frequency')

plt.tight_layout()

fig.suptitle(f"Population density histograms (percent included = {fires_included_prop}%)", fontsize=20)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
pdf_filename = plot_dir / f"histograms.pdf"
plt.savefig(pdf_filename, format='pdf')
plt.close(fig)

In [ ]:
for disaster in tqdm.tqdm(disaster_ids): 
    plot_fires(disaster)

In [ ]:
pdf_dir = plot_dir
pdf_files = sorted(pdf_dir.glob("fire_pop_dens_*.pdf"))
histogram_pdf = pdf_dir / "histograms.pdf"
output_pdf = pdf_dir / "compiled/combined_fire_plots.pdf"
pdf_merger = PyPDF2.PdfMerger()

# histogram first
with open(histogram_pdf, 'rb') as f:
    pdf_merger.append(f)
    
# then loop through each fire's pdf
for pdf_file in pdf_files:
    with open(pdf_file, 'rb') as f:
        pdf_merger.append(f)
with open(output_pdf, 'wb') as output_file:
    pdf_merger.write(output_file)
pdf_merger.close()
print(f"Combined PDF saved as {output_pdf}")

In [ ]:
# # plots 

# def plot_fires(disaster_id): 
#     cmap = "RdGy"
#     df_plot = pd.read_parquet(data_dir / "processed/fire_pop_density_criteria.parquet").iloc[0]
#     df_plot = df[df["disaster_id"] == disaster_id].iloc[0]
#     fire_poly = wkt.loads(df_plot["geometry"])
#     crs = df_plot['crs']
#     fire_series = gpd.GeoSeries([fire_poly], crs=crs)
#     buffer_dist = large_fire_buffer if fire_series.area.iloc[0] > area_thresh else small_fire_buffer 
#     buffered_fire_series = fire_series.buffer(buffer_dist)
#     fire_poly_plus_buffer = gpd.GeoSeries([fire_poly, buffered_fire_series.iloc[0]], crs=crs)
    
#     # gridspec layout 
#     fig = plt.figure(figsize=(20, 15))
#     gs = gridspec.GridSpec(2, 3, figure=fig, width_ratios=[1, 1, 2], height_ratios=[1, 1])
    
#     # fig 1: fire_poly
#     ax1 = fig.add_subplot(gs[0, 0])
#     fire_series.plot(ax=ax1, edgecolor='firebrick', facecolor='none')
#     ax1.set_title(f"Fire Polygon", fontsize=16)
#     ax1.title.set_position([0.5, 1.05])
#     ax1.set_axis_off()
    
#     # fig 2: fire poly with buffer
#     ax2 = fig.add_subplot(gs[0, 1])
#     fire_poly_plus_buffer.boundary.plot(ax=ax2, linewidth=1.5, edgecolor = 'firebrick')
#     ax2.set_title(f"Fire Polygon with Buffer (Buffer Distance: {buffer_dist} meters)", fontsize=16)
#     ax2.title.set_position([0.5, 1.05])
#     ax2.set_axis_off()
    
#     # fig 3: population with fire poly
#     bounding_box = buffered_fire_series.envelope.buffer(pop_density_radius * 1.1).to_crs(mol_crs).iloc[0]
#     pop = rt.load_raster(data_dir / f"raw/pop_data/GHS_POP_E{year}_GLOBE_R2023A_54009_100_V1_0.tif", bounding_box)
#     ax3 = fig.add_subplot(gs[1, 0])
#     pop.plot(ax=ax3, cmap='viridis')
#     fire_series.to_crs(mol_crs).boundary.plot(ax=ax3, color='firebrick')
#     ax3.set_title("Population density and fire polygon", fontsize=16)
#     ax3.set_axis_off()
    
#     # fig 4: pop density in box
#     pop_density = make_spatial_sum(pop, pop_density_radius)
#     ax4 = fig.add_subplot(gs[1, 1])
#     pop_density.plot(ax=ax4, cmap='plasma')
#     ax4.set_title("Population density in bounding box", fontsize=16)
#     ax4.set_axis_off()
    
#     # fig 5: pop density in fire poly 
#     ax5 = fig.add_subplot(gs[:, 2])
#     pop_density.mask(fire_series.to_crs(mol_crs)).plot(ax=ax5, cmap='plasma')
#     ax5.set_title("Population density within fire polygon", fontsize=16)
#     ax5.set_axis_off()
#     ax5.set_xlim(fire_series.to_crs(mol_crs).total_bounds[[0, 2]])
#     ax5.set_ylim(fire_series.to_crs(mol_crs).total_bounds[[1, 3]])
    
#     fig.suptitle(f"Population density criteria for wildfires \nDisaster ID: {df_plot['disaster_id']}, State: {df_plot['state']}, Year: {df_plot['year']} \nCriteria met: {df_plot["density_criteria_met"]}", fontsize=20)
#     plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#     pdf_filename = plot_dir / f"fire_pop_dens_{df_plot['disaster_id']}.pdf"
#     plt.savefig(pdf_filename, format='pdf')

In [ ]:
30/300**2 * 1000**2